### Libraries

In [1]:
import glob
import os
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
import texthero as hero
from texthero import preprocessing
sb.set() # set the default Seaborn style for graphics

### Dataset

In [10]:
steamData = pd.read_csv('steam_data/steam.csv')
steamSupportInfo = pd.read_csv('steam_data/steam_support_info.csv')
steamChart = pd.read_csv('steam_data/steamCharts.csv')
steamSpyTagData = pd.read_csv('steam_data/steamspy_tag_data.csv')
steamGames = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "steam_data/steam_games/steam_games*.csv"))))
steamMediaData = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "steam_data/steam_media_data/steam_media_data*.csv"))))
steamRequirements = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "steam_data/steam_requirements/steam_requirements*.csv"))))
steamDescription = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "steam_data/steam_description/steam_description*.csv"))))

#game reviews
steam_HalfOfTheGameReviews = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "steam_data/steam_reviews/steam_review_*.csv"))))
files = glob.glob("steam_data/steam_reviews" + '**/**' + "/*.csv")
steamReviews_fromFolder = pd.concat([pd.read_csv(f) for f in files ])
steamReviewsDF = [steam_HalfOfTheGameReviews, steamReviews_fromFolder]
steamCombinedReviews = pd.concat(steamReviewsDF)

C:\Users\Pyae Pyae\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (23) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\Pyae Pyae\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (1,23) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\Pyae Pyae\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\Pyae Pyae\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (23) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


### Cleaning the Dataset

In [11]:
#rename for easier identification
steamGames = steamGames.rename(columns={"name": "title"})

#create a custom cleaning pipeline
custom_pipeline = [preprocessing.fillna
                   , preprocessing.lowercase
                   #, preprocessing.remove_digits
                   , preprocessing.remove_punctuation
                   , preprocessing.remove_diacritics
                   #, preprocessing.remove_stopwords
                   , preprocessing.remove_whitespace
                   , preprocessing.stem]

steamCombinedReviews['title'] = hero.clean(steamCombinedReviews['title'], pipeline = custom_pipeline)
steamGames['title'] = hero.clean(steamGames['title'], pipeline = custom_pipeline)

In [12]:
#getting title and genre from steamGames df
steamGames_Title_Genre = steamGames[['title', 'genre']]

#getting title and review from steamCombinedReviews df
steamCombinedReviews_Title_Reviews = steamCombinedReviews[['title', 'review']]

In [13]:
display(steamGames_Title_Genre)

,title,genre
0,doom,Action
1,playerunknown s battleground,"Action,Adventure,Massively Multiplayer"
2,battletech,"Action,Adventure,Strategy"
3,dayz,"Action,Adventure,Massively Multiplayer"
4,eve onlin,"Action,Free to Play,Massively Multiplayer,RPG,..."
...,...,...
828,"rocksmith(r) 2014 edit - remast - sabaton ""gho...","Casual,Simulation"
829,rocksmith(r) 2014 edit - remast - stone templ ...,"Casual,Simulation"
830,fantasi ground quest of doom 4 a midnight coun...,"Indie,RPG,Strategy"
831,mega man x5 sound collect,Action


match title in steamGames with steamCombinedReview

In [14]:
matchTitle = steamGames_Title_Genre.title.isin(steamCombinedReviews_Title_Reviews.title)
#matchTitle = steamCombinedReviews.title.isin(steamGames.title)
#matchTitle
steamMatchTitle = steamGames_Title_Genre[matchTitle]

In [15]:
steamMatchTitle

,title,genre
1,playerunknown s battleground,"Action,Adventure,Massively Multiplayer"
2,battletech,"Action,Adventure,Strategy"
7,human fall flat,"Adventure,Indie"
8,they are billion,"Strategy,Early Access"
42,counter strike sourc,Action
...,...,...
5169,,NaN
9343,transport fever 2,"Simulation,Strategy"
9575,,"Casual,Strategy"
9790,halo the master chief collect,Action


In [8]:
steamUniqueTitle = steamMatchTitle.drop_duplicates(['title'])
steamUniqueTitle

,title,genre
1,playerunknown s battleground,"Action,Adventure,Massively Multiplayer"
2,battletech,"Action,Adventure,Strategy"
7,human fall flat,"Adventure,Indie"
8,they are billion,"Strategy,Early Access"
42,counter strike sourc,Action
...,...,...
1596,atlas,"Action,Adventure,Massively Multiplayer,RPG,Ear..."
1971,metro exodus,Action
9343,transport fever 2,"Simulation,Strategy"
9790,halo the master chief collect,Action


In [9]:
steamUniqueTitle.to_excel("output.xlsx")  